For this assignment, we will be implementing the following paper from scratch!!:

https://arxiv.org/pdf/1501.07873.pdf

Please feel free to read the paper, though you are not required to :) 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


###Image Preprocessing

The only thing that needs to be modified is the file path to our sketches.

Please feel free to change the number of epochs, batch size or learning rate if you wish, though it is not required.

In [ ]:
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import gc
from tqdm import tqdm

from torch.utils.data import DataLoader

from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
def get_sketches():
    TRANSFORM_IMG = transforms.Compose([
        transforms.Resize((225, 225)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    DATA_DIR = "/content/drive/My Drive/MLBootcamp/Week Three/Day Thirteen/sketches" ### MODIFY FILE PATH
    train_data = datasets.ImageFolder(DATA_DIR, transform=TRANSFORM_IMG)
    test_data = datasets.ImageFolder(DATA_DIR, transform=TRANSFORM_IMG)
    num_train = len(train_data)
    indices = list(range(num_train))
    valid_size = .2
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = DataLoader(train_data,
                   sampler=train_sampler, batch_size=64)
    testloader = DataLoader(test_data,
                   sampler=test_sampler, batch_size=64)
    return trainloader, testloader

Feel free to change num_epochs, batch_size and learning rate.

In [ ]:
num_epochs = 40
num_classes = 5 
batch_size = 64
learning_rate = 0.001

print(num_epochs, num_classes, batch_size, learning_rate)

40 5 64 0.001


###Model

Code in this section needs to be written by you!

Take a look at the following link and make your Convolutional Neural network using those specifications:

[Sketch-A-Net Architecture](https://drive.google.com/file/d/1RGQb_KeEAWLXu9sFMVdInMhzt7fDKHvl/view?usp=sharing)

Note 1: Do not modify layer 1 and fc1 (layer 8). Use layer 1 as a template.

Note 2: Only init later needs to be modified in this example. Forward method though does not. Please take a look at the code in the forward method though.

Note 3: https://pytorch.org/docs/master/generated/torch.nn.Conv2d.html

https://pytorch.org/docs/stable/nn.html#dropout-layers

may help!

In [ ]:
class Sketch_a_Net_CNN(torch.nn.Module):
    def __init__(self):
        super(Sketch_a_Net_CNN, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=15, stride=3, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU()
        )

        self.layer5 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=7, stride=1, padding=0),
            nn.ReLU(),
            nn.Dropout()
        )

        self.layer7 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=1, stride=1, padding=0),
            nn.ReLU(),
            nn.Dropout()
        )
        
        self.fc1 = nn.Sequential(
            nn.Linear(512, 5)
        )

         
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        return(out)

###Training and Evaluation

Though you don't need to modify any code in this section, please feel free to uncomment the lines to see the Top 5 Accuracy!

In [ ]:
def train():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = Sketch_a_Net_CNN().to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    train_loader, val_loader = get_sketches()

    total_step = len(train_loader)

    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(tqdm(train_loader), 1):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        torch.save(model.state_dict(), "model.%d" % epoch)

        model.eval()

        train_correct = 0
        train_five_correct = 0
        train_total = 0
        with torch.no_grad():
            for data in tqdm(train_loader):
                images, labels = data
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                _, predicted_five = torch.topk(outputs.data, 5, dim=1)

                train_total += labels.size(0)

                train_correct += (predicted == labels).sum().item()


                #train_five_correct += (predicted_five[:, 0] == labels).sum().item()
                #train_five_correct += (predicted_five[:, 1] == labels).sum().item()
                #train_five_correct += (predicted_five[:, 2] == labels).sum().item()
                #train_five_correct += (predicted_five[:, 3] == labels).sum().item()
                #train_five_correct += (predicted_five[:, 4] == labels).sum().item()
        
        print('Top One Error of the network on train images: %d %%' % (
                100 * (1 - train_correct / train_total)))

        #print('Top Five Error of the network on train images: %d %%' % (
                #100 * (1 - train_five_correct /train_total)))

        correct = 0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for data in tqdm(val_loader):
                images, labels = data

                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                _, predicted_five = torch.topk(outputs.data, 5, dim=1)

                val_total += labels.size(0)

                correct += (predicted == labels).sum().item()

                #val_correct += (predicted_five[:, 0] == labels).sum().item()
                #val_correct += (predicted_five[:, 1] == labels).sum().item()
                #val_correct += (predicted_five[:, 2] == labels).sum().item()
                #val_correct += (predicted_five[:, 3] == labels).sum().item()
                #val_correct += (predicted_five[:, 4] == labels).sum().item()
        

        print('Top One Error of the network on validation images: %d %%' % (
                100 * (1 - correct / val_total)))


        #print('Top Five Error of the network on validation images: %d %%' % (
                #100 * (1 - val_correct / val_total)))
        

        gc.collect()

In [ ]:
train()

  0%|          | 0/2 [00:00<?, ?it/s]

Top One Error of the network on train images: 77 %


  0%|          | 0/5 [00:00<?, ?it/s]

Top One Error of the network on validation images: 90 %


  0%|          | 0/2 [00:00<?, ?it/s]

...

Top One Error of the network on validation images: 16 %


  0%|          | 0/2 [00:00<?, ?it/s]

Top One Error of the network on train images: 0 %


100%|██████████| 2/2 [00:00<00:00,  2.06it/s]

Top One Error of the network on validation images: 17 %
